<a id='top'></a>
- <a href='#A'>A. Préparation des données</a>
	- <a href='#A.1'>A.1. Importation des modules Python</a>
	- <a href='#A.2'>A.2. Chargement du jeu de données</a>
	- <a href='#A.3'>A.3. Nettoyage des données</a>
		- <a href='#A.3.1'>A.3.1. Traitement des valeurs manquantes</a>
		- <a href='#A.3.2'>A.3.2. Encodage des variables catégorielles</a>
		- <a href='#A.3.3'>A.3.3. Suppression des doublons</a>
	- <a href='#A.4'>A.4. Exploration des données</a>
		- <a href='#A.4.1'>A.4.1. Analyse et visualisation des corrélations</a>
		- <a href='#A.4.2'>A.4.2. Visualisation des distributions</a>
		- <a href='#A.4.3'>A.4.3. Normalisation des données</a>
		- <a href='#A.4.4'>A.4.4. Tests statistiques</a>
		- <a href='#A.4.5'>A.4.5. Détection et correction des anomalies: Traitement des valeurs aberrantes</a>
		- <a href='#A.4.6'>A.4.6. Analyse statistique descriptive</a>
	- <a href='#A.5'>A.5. Features Engineering</a>
	- <a href='#A.6'>A.6. Séparation des données</a>
		- <a href='#A.6.1'>A.6.1. Division en ensembles d'apprentissage et de test</a>
		- <a href='#A.6.2'>A.6.2. Localisation de la variable cible Attrition</a>

## <a id='A'>A. Préparation des données </a>

### <a id='A.1'>A.1. Importation des modules Python</a>

In [19]:
import unicodedata
from openpyxl import load_workbook
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

import warnings
warnings.filterwarnings('ignore')

In [20]:
columnsNameNomalizer = lambda x: str(
    unicodedata.normalize('NFKD', x).encode('ascii', 'ignore'
                       ).decode('utf-8')).title().replace('_', '').strip() if '_' in str(x) else str(
    unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8')).strip()

### <a id='A.2'>A.2. Chargement du jeu de données</a>

### A.2.1. Ouverture, normalisation des noms des variables et suppression de variables inutiles pour la prédiction

In [21]:
# Load workbook
raw_data_filepath = "../../data/raw/Exam_ML_ISE-2.xlsx"
wb = load_workbook(filename=raw_data_filepath)
print(f'Nom des feuilles du classeur excel: {wb.sheetnames}')

Nom des feuilles du classeur excel: ['Présentation du Jeux de données', 'Description des Variables', 'JeuxDeDonnées']


In [22]:
# Access to a the worksheet named 'JeuxDeDonnées'
ws = wb[wb.sheetnames[-1]]
# Get the first line in file as a header line
values = ws.values
columns = next(values)[0:]
# Create a DataFrame based on the second and subsequent lines of data
data = pd.DataFrame(values, columns=columns)

In [23]:
# Normalisation des noms des variables: en enlevant les caractères accentués et
# les espaces(Genre) ainsi qu'en renommant de manière uniforme sans underscore
data.rename(columns={c: columnsNameNomalizer(c) for c in data.columns}, inplace=True)

In [24]:
categorical_features = data.select_dtypes(include=['object', 'category']).columns
numerical_features = data.select_dtypes(exclude=['object', 'category']).columns

print(f'''Nombre de variables catégorielles:\t{categorical_features.size}
Nombre de variables quantitatives:\t{numerical_features.size}
----------------------------------\t--
Total à l'ouverture de la base:\t\t{categorical_features.size + numerical_features.size}''')

Nombre de variables catégorielles:	9
Nombre de variables quantitatives:	26
----------------------------------	--
Total à l'ouverture de la base:		35


#### Suppression des variables inutiles

In [25]:
data.Age.min()

18

In [26]:
data.Over18Index.value_counts()

Over18Index
Y    1470
Name: count, dtype: int64

In [27]:
data.StandardHours.value_counts()

StandardHours
80    1470
Name: count, dtype: int64

On remarque que la variable `Over18Index` ne prend qu'une seule valeur car l'age minimum des employés est 18 ans.\
Il ne sert donc à rien de la garder.\
Il en est de même pour la variable `StandardHours` qui ne prend qu'une seule valeure.

De plus, nous allons supprimer les variables `EmployeeCount` et `EmployeeNumber` puisque n'ayant aucun intérêt prédictif mais d'anonymat.

In [28]:
data.drop(columns={'EmployeeCount', 'EmployeeNumber', 'Over18Index', 'StandardHours'}, inplace=True)

In [29]:
categorical_features = data.select_dtypes(include=['object', 'category']).columns
numerical_features = data.select_dtypes(exclude=['object', 'category']).columns

print(f'''Nombre de variables catégorielles:\t{categorical_features.size}
Nombre de variables quantitatives:\t{numerical_features.size}
----------------------------------\t--
Total après chargement de la base:\t{categorical_features.size + numerical_features.size}''')

Nombre de variables catégorielles:	8
Nombre de variables quantitatives:	23
----------------------------------	--
Total après chargement de la base:	31


In [30]:
data.to_csv("../../data/processed/processed1_loading.csv", encoding='utf8', index=False)

### <a id='A.2.2'>A.2.2. Définition des types des variables</a>

Ceci est fait dans le but d'optimiser l'allocation des ressources et de meilleurs performances

In [31]:
dtypes = {
    'Age': 'int8',
    'Attrition': 'category',               # unordered
    #'DailyRate': ,
    'Department': 'category',              # unordered
    #'DistanceFromHome': ,
    'Education': 'int8',#'category',
    'EducationField': 'category',          # unordered
    'EnvironmentSatisfaction': 'int8',#''category',
    'EtatCivil': 'category',               # unordered
    'EvaluationPerformance': 'int8',#''category',
    'Genre': 'category',                   # unordered
    'HeuresSupplementaires': 'category',   # unordered
    'HourlyRate': 'int8',
    'ImplicationDansEmploi': 'int8',#''category',
    'JobLevel': 'int8',#''category',
    'JobRole': 'category',                 # unordered
    #'MonthlyRate',
    'NumCompaniesWorked': 'int8',
    'PercentSalaryHike': 'int8',
    #'RevenuMensuel',
    'SatisfactionRelationnelle': 'int8',#''category',
    'SatisfactionTravail': 'int8',#''category',
    'StockOptionLevel': 'int8',#''category',
    'TotalWorkingYears': 'int8',
    'TrainingTimesLastYear': 'int8',
    'VoyageAffaires': 'category',          # unordered
    'WorkLifeBalance': 'int8',#''category',
    'YearsAtCompany': 'int8',
    'YearsInCurrentRole': 'int8',
    'YearsSinceLastPromotion': 'int8',
    'YearsWithCurrManager': 'int8'
}

In [32]:
data = pd.read_csv(filepath_or_buffer="../../data/processed/processed1_loading.csv", dtype=dtypes,)

> Labelizing the data for graphics

In [33]:
Education_codes = {1: 'inférieur au collège', 2: 'collège', 3: 'licence', 4: 'master', 5: 'docteur'}
EnvironmentSatisfaction_codes = {1: 'faible', 2: 'moyen', 3: 'élevée', 4: 'très élevée'}
EvaluationPerformance_codes = {1: 'faible', 2: 'bon', 3: 'excellent', 4: 'exceptionnel'}
ImplicationDansEmploi_codes = {1: 'très peu impliqué', 2: 'peu impliqué', 3: 'impliqué', 4: 'très impliqué', 5: 'exceptionnellement impliqué'}
JobLevel_codes = {1: 'bas', 2: 'intermédiaire', 3: 'supérieur', 4: 'haut', 5: 'exceptionnel'}
SatisfactionRelationnelle_codes = {1: 'faible', 2: 'moyen', 3: 'élevée', 4: 'très élevée'}
SatisfactionTravail_codes = {1: 'faible', 2: 'moyen', 3: 'élevée', 4: 'très élevée'}
StockOptionLevel_codes = {0: "pas d'option", 1: 'standard', 2: 'élevé', 3: 'exceptionnel '}
WorkLifeBalance_codes = {1: 'mauvais', 2: 'bon', 3: 'excellent', 4: 'très élevé'}
#-------------------------------------------------------------------------------------------------------------------
Education_type = CategoricalDtype(categories=Education_codes.values(), ordered=True)
EnvironmentSatisfaction_type = CategoricalDtype(categories=EnvironmentSatisfaction_codes.values(), ordered=True)
EvaluationPerformance_type = CategoricalDtype(categories=EvaluationPerformance_codes.values(), ordered=True)
ImplicationDansEmploi_type = CategoricalDtype(categories=ImplicationDansEmploi_codes.values(), ordered=True)
JobLevel_type = CategoricalDtype(categories=JobLevel_codes.values(), ordered=True)
SatisfactionRelationnelle_type = CategoricalDtype(categories=SatisfactionRelationnelle_codes.values(), ordered=True)
SatisfactionTravail_type = CategoricalDtype(categories=SatisfactionTravail_codes.values(), ordered=True)
StockOptionLevel_type = CategoricalDtype(categories=StockOptionLevel_codes.values(), ordered=True)
WorkLifeBalance_type = CategoricalDtype(categories=WorkLifeBalance_codes.values(), ordered=True)
#-------------------------------------------------------------------------------------------------------------------
data_labeled = data.copy()
data_labeled['Education'] = data_labeled.Education.map(Education_codes).astype(Education_type)
data_labeled['EnvironmentSatisfaction'] = data_labeled.EnvironmentSatisfaction.map(EnvironmentSatisfaction_codes).astype(EnvironmentSatisfaction_type)
data_labeled['EvaluationPerformance'] = data_labeled.EvaluationPerformance.map(EvaluationPerformance_codes).astype(EvaluationPerformance_type)
data_labeled['ImplicationDansEmploi'] = data_labeled.ImplicationDansEmploi.map(ImplicationDansEmploi_codes).astype(ImplicationDansEmploi_type)
data_labeled['JobLevel'] = data_labeled.JobLevel.map(JobLevel_codes).astype(JobLevel_type)
data_labeled['SatisfactionRelationnelle'] = data_labeled.SatisfactionRelationnelle.map(SatisfactionRelationnelle_codes).astype(SatisfactionRelationnelle_type)
data_labeled['SatisfactionTravail'] = data_labeled.SatisfactionTravail.map(SatisfactionTravail_codes).astype(SatisfactionTravail_type)
data_labeled['StockOptionLevel'] = data_labeled.StockOptionLevel.map(StockOptionLevel_codes).astype(StockOptionLevel_type)
data_labeled['WorkLifeBalance'] = data_labeled.WorkLifeBalance.map(WorkLifeBalance_codes).astype(WorkLifeBalance_type)

In [34]:
data_labeled.head(2)

,Age,Attrition,VoyageAffaires,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Genre,...,EvaluationPerformance,SatisfactionRelationnelle,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1.0,collège,Life Sciences,moyen,Female,...,excellent,faible,pas d'option,8,0,mauvais,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8.0,inférieur au collège,Life Sciences,élevée,Male,...,exceptionnel,très élevée,standard,10,3,excellent,10,7,1,7


In [ ]:
categorical_features = data_labeled.select_dtypes(include='category').columns
ordinal_features = pd.Index(['Education', 'EnvironmentSatisfaction', 'EvaluationPerformance', 'ImplicationDansEmploi', 'JobLevel', 
                             'SatisfactionRelationnelle', 'SatisfactionTravail', 'StockOptionLevel', 'WorkLifeBalance'])
nominal_features = np.setdiff1d(categorical_features, ordinal_features)

numerical_features = data_labeled.select_dtypes(exclude='category').columns

print(f'''Nombre de variables qualitatives ordinales:\t{ordinal_features.size}
Nombre de variables qualitatives nominales:\t{nominal_features.size}
-------------------------------------------\t--
Nombre de variables catégorielles:\t\t{categorical_features.size}
Nombre de variables quantitatives:\t\t{numerical_features.size}
-------------------------------------------\t--
Total après chargement de la base:\t\t{categorical_features.size + numerical_features.size}''')

Nombre de variables qualitatives ordinales:	9
Nombre de variables qualitatives nominales:	8
-------------------------------------------	--
Nombre de variables catégorielles:		17
Nombre de variables quantitatives:		14
-------------------------------------------	--
Total après chargement de la base:		31


In [35]:
data_labeled.to_csv("../../data/processed/processed2_labeled.csv", encoding='utf8', index=False)

### <a id='A.3'>A.3. Nettoyage des données</a>
<a href='#top' style='text-decoration:none; font-family: cursive; '>Back to Top^</a>

#### <a id='A.3.1'>A.3.1. Traitement des valeurs manquantes</a>

In [37]:
#### <a id='A.3.2'>A.3.2. Encodage des variables catégorielles</a>

In [38]:
#### <a id='A.3.3'>A.3.3. Suppression des doublons</a>

In [39]:
### <a id='A.4'>A.4. Exploration des données</a>

In [40]:
#### <a id='A.4.1'>A.4.1. Tests statistiques</a>

In [41]:
#### <a id='A.4.2'>A.4.2. Visualisation des distributions</a>

In [42]:
#### <a id='A.4.3'>A.4.3. Détection et correction des anomalies: Traitement des valeurs aberrantes</a>

In [43]:
#### <a id='A.4.4'>A.4.4. Analyse et visualisation des corrélations</a>

In [44]:
#### <a id='A.4.5'>A.4.5. Normalisation des données</a>

In [45]:
#### <a id='A.4.6'>A.4.6. Analyse statistique descriptive</a>

In [46]:
### <a id='A.5'>A.5. Features Engineering</a>

In [47]:
### <a id='A.6'>A.6. Séparation des données</a>

In [48]:
#### <a id='A.6.1'>A.6.1. Localisation de la variable cible Attrition</a>

In [49]:
#### <a id='A.6.2'>A.6.2. Division en ensembles d'apprentissage et de test</a>